In [1]:
# Dependencies
import pandas as pd
from census import Census
from config import census_api_key

In [2]:
c = Census(census_api_key)

In [3]:
# Run Census Search to retrieve data on all states
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E", "B17001_002E", "B23025_005E"), {'for': 'zip code tabulation area:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "B23025_005E": "Unemployment Count",
                                      "NAME": "Name", "zip code tabulation area": "Zipcode"})

# Add in Poverty Rate (Poverty Count / Population)
census_pd["Poverty Rate"] = 100 * \
    census_pd["Poverty Count"].astype(
        int) / census_pd["Population"].astype(int)

# Add in Employment Rate (Employment Count / Population)
census_pd["Unemployment Rate"] = 100 * \
    census_pd["Unemployment Count"].astype(
        int) / census_pd["Population"].astype(int)

# Final DataFrame
census_pd = census_pd[["Zipcode", "Population", "Median Age", "Household Income",
                       "Per Capita Income", "Poverty Count", "Poverty Rate", "Unemployment Rate"]]

# Visualize
print(len(census_pd))
census_pd.head()

33120


,Zipcode,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Unemployment Rate
0,00601,17599.0,38.9,11757.0,7041.0,11282.0,64.105915,13.943974
1,00602,39209.0,40.9,16190.0,8978.0,20428.0,52.100283,6.473004
2,00603,50135.0,40.4,16645.0,10897.0,25176.0,50.216416,7.156677
3,00606,6304.0,42.8,13387.0,5960.0,4092.0,64.911168,3.236041
4,00610,27590.0,41.4,18741.0,9266.0,12553.0,45.498369,5.342515


In [4]:
# Save as a csv
# To avoid any issues later, we use encoding="utf-8"
census_pd.to_csv("Resources/census_data_zipcodes.csv", encoding="utf-8", index=False)